In [1]:
%run basics
%matplotlib

Using matplotlib backend: Qt4Agg


In [2]:
cf = qcio.load_controlfile(path='controlfiles')
infilename = qcio.get_infilenamefromcf(cf)
ds = qcio.nc_read_series(infilename)
ts = ds.globalattributes["time_step"]
ldt = ds.series["DateTime"]["Data"]
Fc_data,Fc_flag,Fc_attr = qcutils.GetSeriesasMA(ds,"Fc")

In [3]:
ustar,f,a = qcutils.GetSeriesasMA(ds,"ustar")
Fsd,f,a = qcutils.GetSeriesasMA(ds,"Fsd")
if "solar_altitude" not in ds.series.keys(): qcts.get_synthetic_fsd(ds)
Fsd_syn,f,a = qcutils.GetSeriesasMA(ds,"Fsd_syn")
sa,f,a = qcutils.GetSeriesasMA(ds,"solar_altitude")

In [4]:
indicators = {}
ustar_dict = qcrp.get_ustar_thresholds(cf,ldt)
indicators["turbulence"] = qcrp.get_turbulence_indicator_ustar(ldt,ustar,ustar_dict,ts)
indicator = numpy.copy(indicators["turbulence"]["values"])

In [5]:
indicators["day"] = qcrp.get_day_indicator(cf,Fsd,Fsd_syn,sa)
opt = qcutils.get_keyvaluefromcf(cf,["Options"],"AcceptDayTimes",default="Yes")
if opt.lower()=="yes":
    idx = numpy.where(indicators["day"]["values"]==1)[0]
    indicator[idx] = numpy.int(1)

In [6]:
fig = plt.figure(figsize=(24,6))
plt.plot(ldt,Fsd,'bo')
plt.plot(ldt,Fsd_syn,'go')
plt.plot(ldt,indicators["day"],'r+')
plt.tight_layout()
plt.show()

In [7]:
indicators["evening"] = qcrp.get_evening_indicator(cf,Fsd,Fsd_syn,sa,ts)
indicators["dayevening"] = indicators["day"]+indicators["evening"]
opt = qcutils.get_keyvaluefromcf(cf,["Options"],"UseEveningFilter",default="Yes")
if opt.lower()=="yes":
    idx = numpy.where(indicators["dayevening"]==0)[0]
    indicator[idx] = numpy.int(0)

In [8]:
fig=plt.figure(figsize=(24,6))
ax1 = plt.subplot(511)
plt.plot(ldt,indicators["turbulence"],'b.')
plt.ylim([-0.5,1.5])
ax2 = plt.subplot(512,sharex=ax1)
plt.plot(ldt,indicators["day"],'b.')
plt.ylim([-0.5,1.5])
ax3 = plt.subplot(513,sharex=ax1)
plt.plot(ldt,indicators["evening"],'b.')
plt.ylim([-0.5,1.5])
ax4 = plt.subplot(514,sharex=ax1)
plt.plot(ldt,indicators["dayevening"],'b.')
plt.ylim([-0.5,1.5])
ax5 = plt.subplot(515,sharex=ax1)
plt.plot(ldt,indicator,'b.')
plt.ylim([-0.5,1.5])
plt.tight_layout()
plt.show()